# Plot the histogram distribution graphs together
# classes = train.class_name.unique() contains the names of different classes 
fig, axs = plt.subplots(3, 4, figsize = (15, 20)) # 5 rows and 3 columns
fig.subplots_adjust(hspace = .1, wspace = .1)
axs = axs.ravel()
fig = plt.figure(figsize=(15,3))
fig.subplots_adjust(hspace = .1, wspace = .1)
fig.suptitle("Histogram Distribution")
n = 0
for i in range(3):
    fig = plt.figure(figsize=(15,3))
    fig.subplots_adjust(hspace = .1, wspace = .1)
    fig.suptitle("Histogram Distribution")
    for j in range(4):
        n += 1
        fig.add_subplot(i+1, j+1, i+1)
        plt.plot(DrawGauss(n))
        plt.title(classes[n])
    plt.show()
    
#plt.show()

# This snippet of code is from 
# https://stackoverflow.com/questions/19695249/load-just-part-of-an-image-in-python/19772446#19772446
# Since it contains the usage of Pool, I keep the link for future use.
#import time
"""
from PIL import Image
from multiprocessing import Pool

# crop out the part of the whole image to process further analysis
def crop_image(dir, edge):
    im = Image.open(dir)
    w,h = im.size
    imc = im.crop(w-edge[3], h-edge[2], w-edge[1], h-edge[0])
    return np.array(imc)
 """   


## Notebooks and webpages I referenced
[VinBigData - Crop and Resize X-Ray Images](https://www.kaggle.com/anhlv2312/vinbigdata-crop-and-resize-x-ray-images)

The procedure of cropping is not necessary in my opinion. However, the idea of using **brightness scale** is a good idea to me. I may use that in the future code.

[Creating WHL file - Retinanet](https://www.kaggle.com/akhileshdkapse/creating-whl-file-retinanet). This one tells how to use RetinalNet to do the classification job.

[Visual In-Depth EDA – VinBigData Competition Data](https://www.kaggle.com/dschettler8845/visual-in-depth-eda-vinbigdata-competition-data) Heat map is one of the attracting point in this notebook. I can learn how to draw sub-plot from here

[Localization of Findings](https://www.kaggle.com/craigmthomas/localization-of-findings) Another notebook about heatmap. I can learn more skills on sub_plot from this notebook.

[VBD EfficientDET TF2 Object Detection API](https://www.kaggle.com/sreevishnudamodaran/vbd-efficientdet-tf2-object-detection-api) TensorFlow 2 Object Detection API is a framework built on top of TF2, which makes it easy to construct, train and deploy, robust and high performance Object Detection Models. 

[(Python) utility to convert medical images to jpg and png](https://pypi.org/project/med2image/) This a link, descriping the usage of med2image 2.2.10

[Pandas dataframe filter with Multiple conditions](https://kanoki.org/2020/01/21/pandas-dataframe-filter-with-multiple-conditions/)

[Pandas how to get a cell value and update it](https://kanoki.org/2019/04/12/pandas-how-to-get-a-cell-value-and-update-it/)

[Fitter used to find out the best fitting function](https://pypi.org/project/fitter/)

[Applying lambda functions to pandas dataframe](https://www.geeksforgeeks.org/applying-lambda-functions-to-pandas-dataframe/)

[Fitting a 2D gaussian](https://scipy-cookbook.readthedocs.io/items/FittingData.html)

[Plot 2-D Histogram in Python using Matplotlib](https://www.geeksforgeeks.org/plot-2-d-histogram-in-python-using-matplotlib/)

[Creating a 2d histogram from a numpy matrix](https://stackoverflow.com/questions/27156381/python-creating-a-2d-histogram-from-a-numpy-matrix)

[How to get weighted random choice in Python?](https://www.geeksforgeeks.org/how-to-get-weighted-random-choice-in-python/)

[Reorder pandas dataframe columns](https://www.datasciencemadesimple.com/re-arrange-or-re-order-the-column-of-dataframe-in-pandas-python-2/)
The next page is about reordering rows in a dataframe

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
 
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install mahotas

In [ ]:
import mahotas as mh
import pandas as pd
from fastai.medical.imaging import *
from fastai.vision.all import *
from pathlib import Path
import matplotlib.pyplot as plt
import pydicom # https://pydicom.github.io/ dealing with Dicom (Digital Imaging in Medicine)
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from scipy.optimize import curve_fit
from random import randint
from pylab import gray, imshow, show
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
import warnings
warnings.filterwarnings("ignore", message="The (0028,0101) 'Bits Stored' value (12-bit) doesn't match the JPEG 2000 data (16-bit).")
import time

In [ ]:
dataset_dir = '../input/vinbigdata-chest-xray-abnormalities-detection'
my_dataset = '../input/withhf'

In [ ]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

In [ ]:
def plot_imgs(imgs, cols=4, size=7, is_rgb=True, title="", cmap='gray', img_size=(500,500)):
    rows = len(imgs)//cols + 1
    fig = plt.figure(figsize=(cols*size, rows*size))
    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        fig.add_subplot(rows, cols, i+1)
        plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    #plt.savefig('temp.png')
    plt.show()
    

In [ ]:
def get_area(row):
    return (row['x_max'] - row['x_min']) * (row['y_max'] - row['y_min'])
def center_x(row):
    return (row['x_max'] + row['x_min']) / 2
def center_y(row):
    return (row['y_max'] + row['y_min']) / 2

In [ ]:
def draw_center(num):
    studyClasses = train[train.class_name == classes[num]]
    studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
    plt.figure(figsize=(8,8))
    center = pd.DataFrame()
    center['center_x'] = studyClasses.apply(center_x, axis = 1)
    center['center_y'] = studyClasses.apply(center_y, axis = 1)
    #plt.figure(figsize=(14,6))
    plt.title(classes[num])
    plt.scatter(center.center_x, center.center_y)
    plt.show()

In [ ]:
#!pip install fitter

In [ ]:
#from fitter import Fitter

In [ ]:
train = pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
train.head()
#train.loc[: , ['class_name', 'class_id']]
#train.loc[0,:]

In [ ]:
train.loc[50:80 , ['image_id', 'class_name', 'class_id']]

In [ ]:
train[train['image_id'].str.contains("3a4407a2df891526e94ba4541023f49")]

In [ ]:
#dfObj[dfObj['x_min']> 1300]
dfObj

In [ ]:
dfData[53,:]

In [ ]:
classes = train.class_name.unique()
classes

studyClasses = train[train.class_name == 'Pleural effusion']
center = pd.DataFrame()
center['center_x'] = studyClasses.apply(center_x, axis = 1)
center['center_y'] = studyClasses.apply(center_y, axis = 1)
x = center.center_x
x_min = x.min()
x_max = x.max()
print("x_min, x_max: ", x_min, x_max)

y = center.center_y
y_min = y.min()
y_max = y.max()
print("y_min, y_max: ", y_min, y_max)


x_bins = np.linspace(x_min, x_max, 40) 
y_bins = np.linspace(y_min, y_max, 40)


fig, ax = plt.subplots(figsize =(10, 7)) 
# Creating plot 
plt.hist2d(x, y, bins =[x_bins, y_bins]) 
plt.title("Changing the bin scale")

ax.set_xlabel('X-axis')  
ax.set_ylabel('Y-axis') 

plt.show() 

In [ ]:
studyClasses = train[train.class_id == 8]
center = pd.DataFrame()
center['center_x'] = studyClasses.apply(center_x, axis = 1)
center['center_y'] = studyClasses.apply(center_y, axis = 1)
studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
x = center.center_x
x_min = x.min()
x_max = x.max()

y = center.center_y
y_min = y.min()
y_max = y.max()

z = studyClasses.boxArea

cut = 21
xedges, yedges = np.linspace(x_min, x_max, cut), np.linspace(y_min,  y_max, cut)
hist, xedges, yedges = np.histogram2d(x, y, (xedges, yedges))
xidx = np.clip(np.digitize(x, xedges), 0, hist.shape[0]-1)
yidx = np.clip(np.digitize(y, yedges), 0, hist.shape[1]-1)
c = hist[xidx, yidx]
plt.scatter(x, y, c=c)

plt.show()

In [ ]:
prob = hist/hist.sum()
weight = prob.flatten()
consecut = np.arange(0, len(weight))

select = int(np.count_nonzero(hist.flatten())*0.618)
#select = 100
randomNumberList = np.random.choice(consecut, select, p=weight) 
"""
print(randomNumberList)
print(weight[randomNumberList[5]])
xx = randomNumberList[5]//(cut - 1)
yy = randomNumberList[5] % (cut - 1)
print(prob[xx, yy])
print(xedges[cut-1])
print(yedges[cut-1])
#print(x_min, x_max)
temp = 0
temp_st_x = []
temp_st_y = []
temp_en_x = []
temp_en_y = []
for c, d, st_x, st_y, en_x, en_y in zip(x, y, studyClasses.x_min, studyClasses.y_min, studyClasses.x_max, studyClasses.y_max):
    if ((c > xedges[xx]) & (c < xedges[xx+1])) & ((d > yedges[yy]) & (d < yedges[yy+1])):
        temp_st_x.append(st_x)
        temp_st_y.append(st_y)
        temp_en_x.append(en_x)
        temp_en_y.append(en_y)
        temp += 1
print(temp)  
print(temp_st_x)
"""
# I decide to use the average value to present the value of the selected samples in a grid
#aver_x_min = np.mean(temp_st_x)
#aver_y_min = np.mean(temp_st_y)
#aver_x_max = np.mean(temp_en_x)
#aver_y_max = np.mean(temp_en_y)
aver_x_min = []
aver_y_min = []
aver_x_max = []
aver_y_max = []
for element in randomNumberList:
    xx = element//(cut - 1)
    yy = element % (cut - 1)
    temp_st_x = []
    temp_st_y = []
    temp_en_x = []
    temp_en_y = []
    for c, d, st_x, st_y, en_x, en_y in zip(x, y, studyClasses.x_min, studyClasses.y_min, studyClasses.x_max, studyClasses.y_max):
        if ((c > xedges[xx]) & (c < xedges[xx+1])) & ((d > yedges[yy]) & (d < yedges[yy+1])):
            temp_st_x.append(st_x)
            temp_st_y.append(st_y)
            temp_en_x.append(en_x)
            temp_en_y.append(en_y)
    aver_x_min.append(np.mean(temp_st_x))
    aver_y_min.append(np.mean(temp_st_y))
    aver_x_max.append(np.mean(temp_en_x))
    aver_y_max.append(np.mean(temp_en_y))
    
#print(aver_x_min)
print(len(aver_x_min))

In [ ]:
# The following function is adapted from the function getHF_each_box
# For each img, the function will return a DataFrame for testing
#dfObj = pd.DataFrame()
def getHF_each_image(row):
    #dfObj = pd.DataFrame(columns=row.columns.values)
    imgT = []
    #print(row['image_id'])
    img_id = str(row['image_id'])
    img_path = f'{dataset_dir}/train/{img_id}.dicom'
    img_path = '../input/vinbigdata-chest-xray-abnormalities-detection/test/83caa8a85e03606cf57e49147d7ac569.dicom'
    imgT = dicom2array(path=img_path)
    imgT = np.stack([imgT, imgT, imgT], axis=-1)
    #box = row.loc[['x_min', 'y_min', 'x_max', 'y_max']].values
    #print(box.shape)
    #img = imgT[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
    #meanValue = mh.features.haralick(img).mean(0)
    #get.loc[row.name,['HF1', 'HF2','HF3','HF4','HF5','HF6','HF7','HF8','HF9', 'HF10', 'HF11', 'HF12', 'HF13']] = meanValue
    i = 0
    for x_min, y_min, x_max, y_max in zip(aver_x_min, aver_y_min, aver_x_max, aver_y_max):
        img = imgT[int(x_min):int(x_max), int(y_min):int(y_max)]
        try:
            meanValue = mh.features.haralick(img).mean(0)
            dfObj.loc[row.name+i,['HF1', 'HF2','HF3','HF4','HF5','HF6','HF7','HF8','HF9', 'HF10', 'HF11', 'HF12', 'HF13']] = meanValue
            dfObj.loc[row.name+i,['image_id','x_min', 'y_min', 'x_max', 'y_max']] = row.image_id, x_min, y_min, x_max, y_max
            i += 1
        except:
            print("Some issue happens on mahotas")
            pass
    return dfObj

In [ ]:
# The following lines of codes are making use of 
T1 = time.time()
dfObj = pd.DataFrame(columns=train.columns.values)
dfObj = getHF_each_image(train.loc[56,:])
T2 = time.time()
print ("Time used: ", T2 - T1)
dfObj

In [ ]:
#dfObj.loc[59:72,:]

In [ ]:
train_HF[train_HF['image_id'].str.contains("d106ec9b305178f3da060efe3191499a")]

In [ ]:
boxArea3d(10)

In [ ]:
#According to different given number i, the returned DataFrame will give training data with binary label
train_HF = pd.read_csv('../input/withhf/FoundSthWithHF.csv')
def returnTrainData(i):
    trainData = pd.DataFrame()
    trainData = train_HF
    trainData['chk'] = train_HF.apply(lambda n: 1 if n['class_id'] == i else 0, axis = 1)
    
    #trainData['boxArea'] = train_7.apply(get_area, axis = 1)   
    #trainData['xCenter'] = train_7.apply(center_x, axis = 1)
    #trainData['yCenter'] = train_7.apply(center_y, axis = 1)
    
    return trainData

In [ ]:
train_1['chk'] = train_1.apply(lambda n: 1 if n['class_id'] == 1 else 0, axis = 1)
train_1['boxArea'] = train_1.apply(get_area, axis = 1)   # for now, I will keep those three features so that I can compare current result with previous one
train_1['xCenter'] = train_1.apply(center_x, axis = 1)
train_1['yCenter'] = train_1.apply(center_y, axis = 1)

In [ ]:
train_2['chk'] = train_2.apply(lambda n: 1 if n['class_id'] == 2 else 0, axis = 1)
train_2['boxArea'] = train_1.apply(get_area, axis = 1)   # for now, I will keep those three features so that I can compare current result with previous one
train_2['xCenter'] = train_1.apply(center_x, axis = 1)
train_2['yCenter'] = train_1.apply(center_y, axis = 1)

In [ ]:
train_13.loc[65:85, ['class_id', 'chk']]

In [ ]:
#features = ['x_min', 'y_min', 'x_max', 'y_max', 'HF1', 'HF2', 'HF3', 'HF4',
#       'HF5', 'HF6', 'HF7', 'HF8', 'HF9', 'HF10', 'HF11', 'HF12', 'HF13', 'boxArea', 'xCenter', 'yCenter']
features = ['HF1', 'HF2', 'HF3', 'HF4', 'HF5', 'HF6', 'HF7', 'HF8', 'HF9', 'HF10', 'HF11', 'HF12', 'HF13']

In [ ]:
# Split the given data into features and target, so that we can standardize features and further performing PCA upon those features
inputData = pd.DataFrame()
inputData = returnTrainData(8)
#x = train_0.loc[:, features].values
x = inputData.loc[:, features].values
#y = train_0.loc[:, 'chk'].values
y = inputData.loc[:, 'chk'].values

#x = StandardScaler().fit_transform(x)

In [ ]:
x[0,:]

In [ ]:
pca = PCA(.98)
pca.fit(x)
xPCA = pca.transform(x)

In [ ]:
xPCA[0,:]

In [ ]:
#principalDF = pd.DataFrame(data = xPCA, columns = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10'])
#principalDF = pd.DataFrame(data = xPCA, columns = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7'])
principalDF = pd.DataFrame(data = x, columns = features)

In [ ]:
targetDF = pd.DataFrame(data = dfObj.loc[:, features].values, columns = features)

In [ ]:
dfFinal = pd.concat([principalDF, inputData[['chk']]], axis = 1)

In [ ]:
#dfData = dfFinal.loc[:, ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10']].values
#dfData = dfFinal.loc[:, ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']].values
dfData = dfFinal.loc[:, features].values
dfLabel = dfFinal.loc[:, 'chk'].values

In [ ]:
HFtrain_data, HFtest_data, HFtrain_label, HFtest_label = train_test_split(dfData, dfLabel, test_size=0.2, random_state=0, shuffle=True)

In [ ]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(solver = 'lbfgs')
#logisticRegr.fit(HFtrain_data, HFtrain_label)
#logisticRegr.score(HFtest_data, HFtest_label)
logisticRegr.fit(dfData, dfLabel)

In [ ]:
np.count_nonzero(logisticRegr.predict(dfObj.loc[:, features].values))

In [ ]:
logisticRegr.predict(dfObj.loc[:, features].values)

In [ ]:
#HFtest_label[0:7]

In [ ]:
from sklearn.neural_network import MLPClassifier
clfNN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 8), random_state=0)
#clf.fit(HFtrain_data, HFtrain_label)
clfNN.fit(dfData, dfLabel)
#clf.score(HFtest_data, HFtest_label)

In [ ]:
clfNN.predict((dfObj.loc[:, features].values))

In [ ]:
from sklearn import svm
clfSVM = svm.SVC()
#clf.fit(HFtrain_data, HFtrain_label)
clfSVM.fit(dfData, dfLabel)
#clf.score(HFtest_data, HFtest_label)

In [ ]:
clfSVM.predict((dfObj.loc[:, features].values))

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(KNeighborsClassifier(), max_samples=0.9, max_features=0.9)
#bagging = bagging.fit(HFtrain_data, HFtrain_label)
#bagging.score(HFtest_data, HFtest_label)
bagging = bagging.fit(dfData, dfLabel)

In [ ]:
bagging.predict((dfObj.loc[:, features].values))

In [ ]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=150)
clf.fit(HFtrain_data, HFtrain_label)

In [ ]:
clf.predict((dfObj.loc[:, features].values))

In [ ]:
clf.score(HFtest_data, HFtest_label)

In [ ]:
from sklearn.neighbors import NearestCentroid
clf = NearestCentroid()
clf.fit(HFtrain_data, HFtrain_label)

In [ ]:
clf.score(HFtest_data, HFtest_label)

In [ ]:
from sklearn.neighbors import (NeighborhoodComponentsAnalysis, KNeighborsClassifier)
from sklearn.pipeline import Pipeline
nca = NeighborhoodComponentsAnalysis(random_state=0)
knn = KNeighborsClassifier(n_neighbors=13)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(HFtrain_data, HFtrain_label)

In [ ]:
print(nca_pipe.score(HFtest_data, HFtest_label))

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(HFtrain_data, HFtrain_label)

In [ ]:
gnb.score(HFtest_data, HFtest_label)

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(HFtrain_data, HFtrain_label)

In [ ]:
clf.score(HFtest_data, HFtest_label)

In [ ]:
clf = tree.DecisionTreeRegressor()
clf = clf.fit(HFtrain_data, HFtrain_label)

In [ ]:
clf.score(HFtest_data, HFtest_label)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=40)
clf = clf.fit(HFtrain_data, HFtrain_label)

In [ ]:
clf.score(HFtest_data, HFtest_label)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=100)
clf = clf.fit(HFtrain_data, HFtrain_label)
clf.score(HFtest_data, HFtest_label)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
clf.fit(HFtrain_data, HFtrain_label)
clf.score(HFtrain_data, HFtrain_label)

In [ ]:
testDCM = Path('../input/vinbigdata-chest-xray-abnormalities-detection/train/0032c6091dc8f1b1245fc2f5f45458fa.dicom')
dcm = testDCM.dcmread()
dcm

In [ ]:
cols = ['#e41a1c', '#377eb8','#4daf4a','#984ea3','#ff7f00','#ffff33','#a65628','#f781bf','#999999', '#000000', '#1b9e77', '#d95f02', '#7570b3', '#e7298a']

In [ ]:
dcm.show()

In [ ]:
#train[train['image_id']=='0032c6091dc8f1b1245fc2f5f45458fa']

In [ ]:
le = preprocessing.LabelEncoder() # encode rad_id
train['rad_label'] = le.fit_transform(train['rad_id'])
train.head()

In [ ]:
# Define model function to be used to fit to the data above:
def gauss(x, *p):
    A, mu, sigma = p
    return A*np.exp(-(x-mu)**2/(2.*sigma**2))
# Fucntion to calculate the exponential with constants a and b
def expon(x, a, b):
    return a*np.exp(b*x)
# Define 2d-Guassian function to fit to the data given:
def gauss2d(*p):
    height, center_x, center_y, width_x, width_y = p
    width_x = float(width_x)
    width_y = float(width_y)
    return lambda x,y: height*np.exp(
                -(((center_x-x)/width_x)**2+((center_y-y)/width_y)**2)/2)
# Based on the function above, we need to further define a two-centered Gaussian fucntion
def twoCenterGaussion(*p):
    height1, center_x1, center_y1, width_x1, width_y1, height2, center_x2, center_y2, width_x2, width_y2 = p
    p1 = np.array(height1, center_x1, center_y1, width_x1, width_y1)
    p2 = np.array(height2, center_x2, center_y2, width_x2, width_y2)
    return gauss2d(*p1) + gauss2d(*p2)

# Fitting Gauss
def fitgaussian(data, *pa):
    params = pa
    errorfunction = lambda p: np.ravel(gaussian(*p)(*np.indices(data.shape)) -
                                 data)
    p, success = optimize.leastsq(errorfunction, params)
    return p

In [ ]:
def DrawGauss(num):
    # num is an integer, indicating the positon of classes
    studyClasses = train[train.class_name == classes[num]]
    studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
    y = studyClasses.boxArea
    #plt.figure(figsize=(14,6))
    plt.title(classes[num])
    plt.hist(y, bins = 200)
    mean=y.mean()
    highest=y.max()
    p0 = [highest, mean, mean]
    try:
        hist, bin_edges =  np.histogram(y, bins = 200, density=False)
        bin_centres = (bin_edges[:-1] + bin_edges[1:])/2
        coeff, var_matrix = curve_fit(gauss, bin_centres, hist, p0=p0)
        # Get the fitted curve
        hist_fit = gauss(bin_centres, *coeff)
        plt.plot(bin_centres, hist_fit, label='Fitted data', color='red')
        #print ('Fitted mean = ', coeff[1])
        #print ('Fitted standard deviation = ', coeff[2])
    except:
        print('Mean value = ', mean)
    imag = plt.show()
    #return imag

In [ ]:
# Take a look at the spatial distribution of the area of boxes for different
# categories of diseases
def boxArea3d(num):
    sns.set(style = 'darkgrid')

    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection = '3d')
    studyClasses = train[train.class_name == classes[num]]
    studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
    #plt.figure(figsize=(16,16))
    center = pd.DataFrame()
    center['center_x'] = studyClasses.apply(center_x, axis = 1)
    center['center_y'] = studyClasses.apply(center_y, axis = 1)
    #plt.figure(figsize=(14,6))
    plt.title(classes[num] + ": 3d")
    #plt.scatter(center.center_x, center.center_y)
    ax.set_xlabel("center_x")
    ax.set_ylabel("center_y")
    ax.set_zlabel("boxArea")
    ax.scatter(center.center_x, center.center_y, studyClasses.boxArea)
    fig.show()
    """
    # Let me try plotly to draw interactive graph
    py.init_notebook_mode(connected=True)
    studyClasses = train[train.class_name == classes[num]]
    studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
    center = pd.DataFrame()
    center['center_x'] = studyClasses.apply(center_x, axis = 1)
    center['center_y'] = studyClasses.apply(center_y, axis = 1)
    trace = go.Scatter3d(
        x = center.center_x,
        y = center.center_y,
        z = studyClasses.boxArea,
        mode = 'markers',
        marker = dict(
            size = 12,
            line = dict(
                color = 'rgba(217, 217, 217, 0.14)',
                width = 0.5
            ),
            opacity = 1
        ),
        name = classes[num] + ": 3d"
    )
    data = [trace]
    layout = go.Layout(
        margin = dict(
            l = 0,
            r = 0,
            b = 0,
            t = 0
        )
    )
    fig = go.Figure(data = data, layout = layout)
    py.iplot(fig)

In [ ]:
n = 0
for row in train.iloc[0:-1, 1]:
    if row == "Cardiomegaly":
        n += 1
print(n)

In [ ]:
studyClasses = train[train.class_name == classes[1]]
studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
x = np.arange(0, n, 1)
y = studyClasses.boxArea
plt.figure(figsize=(14,6))
plt.scatter(x, y)
plt.show()

In [ ]:
DrawGauss(1)

In [ ]:
draw_center(1)

In [ ]:
n = 0
for row in train.iloc[0:-1, 1]:
    if row == "Aortic enlargement":
        n += 1
print(n)

In [ ]:
studyAortic = train[train.class_name == "Aortic enlargement"]
studyAortic['boxArea'] = studyAortic.apply(get_area, axis = 1)

In [ ]:
studyAortic.boxArea

In [ ]:
x = np.arange(0, 7162, 1)
y = studyAortic.boxArea
plt.figure(figsize=(14,6))
plt.scatter(x, y)
plt.show()

In [ ]:
n = 0
for row in train.iloc[0:-1, 1]:
    if row == 'Pleural thickening':
        n += 1
print(n)

In [ ]:
studyPleural_thickening = train[train.class_name == "Pleural thickening"]
studyPleural_thickening['boxArea'] = studyPleural_thickening.apply(get_area, axis = 1)

In [ ]:
x = np.arange(0, n, 1)
y = studyPleural_thickening.boxArea
plt.figure(figsize=(14,6))
plt.scatter(x, y)
plt.show()

In [ ]:
studyClasses = train[train.class_name == classes[2]]
studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
y = studyClasses.boxArea
DrawGauss(2)

In [ ]:
draw_center(2)

In [ ]:
plt.figure(figsize=(8,8))
center = pd.DataFrame()
center['center_x'] = studyClasses.apply(center_x, axis = 1)
center['center_y'] = studyClasses.apply(center_y, axis = 1)
plt.figure(figsize=(14,6))
plt.scatter(center.center_x, center.center_y)
plt.show()

In [ ]:
studyClasses = train[train.class_name == classes[3]]
studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
y = studyClasses.boxArea
DrawGauss(3)

In [ ]:
draw_center(3)

In [ ]:
studyClasses = train[train.class_name == classes[4]]
studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
y = studyClasses.boxArea
DrawGauss(4)

In [ ]:
draw_center(4)

In [ ]:
studyClasses = train[train.class_name == classes[5]]
studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
y = studyClasses.boxArea
DrawGauss(5)

In [ ]:
draw_center(5)

In [ ]:
studyClasses = train[train.class_name == classes[6]]
studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
y = studyClasses.boxArea
DrawGauss(6)

In [ ]:
draw_center(6)

In [ ]:
studyClasses = train[train.class_name == classes[7]]
studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
y = studyClasses.boxArea
DrawGauss(7)

In [ ]:
draw_center(7)

In [ ]:
studyClasses = train[train.class_name == classes[8]]
studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
y = studyClasses.boxArea
DrawGauss(8)

In [ ]:
draw_center(8)

In [ ]:
studyClasses = train[train.class_name == classes[9]]
studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
y = studyClasses.boxArea
DrawGauss(9)

In [ ]:
#draw_center(9)

In [ ]:
studyClasses = train[train.class_name == classes[10]]
studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
y = studyClasses.boxArea
DrawGauss(10)

In [ ]:
draw_center(10)

In [ ]:
studyClasses = train[train.class_name == classes[11]]
studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
y = studyClasses.boxArea
DrawGauss(11)

In [ ]:
#draw_center(11)

In [ ]:
studyClasses = train[train.class_name == classes[12]]
studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
y = studyClasses.boxArea
DrawGauss(12)

In [ ]:
draw_center(12)

In [ ]:
studyClasses = train[train.class_name == classes[13]]
studyClasses['boxArea'] = studyClasses.apply(get_area, axis = 1)
y = studyClasses.boxArea
DrawGauss(13)

The following codes are modified from notebook: [EDA-VinBigData Chest X-ray Abnormalities](https://www.kaggle.com/trungthanhnguyen0502/eda-vinbigdata-chest-x-ray-abnormalities)

In [ ]:
FoundSth = train[train['class_name'] != 'No finding']

imgs = []
img_ids = FoundSth['image_id'].values
class_ids = FoundSth['class_id'].unique()

# map label_id to specify color
label2color = {class_id:[randint(0,255) for i in range(3)] for class_id in class_ids}
thickness = 3
scale = 5

for i in range(1):
    img_id = random.choice(img_ids)
    #print(img_id)
    img_path = f'{dataset_dir}/train/{img_id}.dicom'
    img = dicom2array(path=img_path)
    #print("Before resize: ")
    #img
    img = cv2.resize(img, None, fx=1/scale, fy=1/scale)
    #print("After resize: ")
    #img
    img = np.stack([img, img, img], axis=-1)
    
    boxes = FoundSth.loc[FoundSth['image_id'] == img_id, ['x_min', 'y_min', 'x_max', 'y_max']].values/scale
    labels = FoundSth.loc[FoundSth['image_id'] == img_id, ['class_id']].values.squeeze()
    #print(boxes)
    for label_id, box in zip(labels, boxes):
        color = label2color[label_id]
        img = cv2.rectangle(
            img,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            color, thickness
    )
    img = cv2.resize(img, (500,500))
    imgs.append(img)
    
plot_imgs(imgs, cmap=None)

## [Mahotas – Cropping Image](https://www.geeksforgeeks.org/mahotas-cropping-image/?ref=rp)

[Mahotas Features](https://mahotas.readthedocs.io/en/latest/mahotas-features.html)

[数据科学与机器学习](https://mlhowto.readthedocs.io/en/latest/opencv.html)

In [ ]:
HFs = ['HF1', 'HF2','HF3','HF4','HF5','HF6','HF7','HF8','HF9', 'HF10', 'HF11', 'HF12', 'HF13']
#for newcol in HFs:
#    FoundSth[newcol] = 0
if 'HF1' not in FoundSth.columns:
    for newcol in HFs:
        FoundSth[newcol] = 0
#FoundSth.describe()

In [ ]:
#get Haralick Features
def getHF(img_ids):  
    imgT = []
    HF = []
    img_path = f'{dataset_dir}/train/{img_id}.dicom'
    imgT = dicom2array(path=img_path)
    imgT = np.stack([imgT, imgT, imgT], axis=-1)
    
    boxes = FoundSth.loc[FoundSth['image_id'] == img_id, ['x_min', 'y_min', 'x_max', 'y_max']].values
    for box in boxes:
        img = imgT[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
        meanValue = mh.features.haralick(img).mean(0)
        HF.append(meanValue)
    return HF                             

In [ ]:
FoundSth.head()

In [ ]:
# get Haralick Features for a single box
"""
def getHF_each_box(index):
    imgT = []
    HF = []
    img_id = FoundSth.loc[FoundSth.index == index, 'image_id'].values
    img_path = f'{dataset_dir}/train/{img_id[0]}.dicom'
    imgT = dicom2array(path=img_path)
    imgT = np.stack([imgT, imgT, imgT], axis=-1)
    #temp = train[train.index == index]
    box = FoundSth.loc[FoundSth.index == index, ['x_min', 'y_min', 'x_max', 'y_max']].values
    img = imgT[int(box[0,1]):int(box[0,3]), int(box[0,0]):int(box[0,2])]
    meanValue = mh.features.haralick(img).mean(0)
    HF.append(meanValue)
    return HF
"""
#HF
get = FoundSth.loc[6016*3:6016*4]
def getHF_each_box(row):
    imgT = []
    #print(row['image_id'])
    img_id = str(row['image_id'])
    img_path = f'{dataset_dir}/train/{img_id}.dicom'
    imgT = dicom2array(path=img_path)
    imgT = np.stack([imgT, imgT, imgT], axis=-1)
    box = row.loc[['x_min', 'y_min', 'x_max', 'y_max']].values
    #print(box.shape)
    img = imgT[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
    meanValue = mh.features.haralick(img).mean(0)
    get.loc[row.name,['HF1', 'HF2','HF3','HF4','HF5','HF6','HF7','HF8','HF9', 'HF10', 'HF11', 'HF12', 'HF13']] = meanValue
    #print(type(meanValue[0]))
    #row.descript
    #get.iloc[row.index:row.index,'HF1'] = meanValue[0]
    #HF.append(meanValue)
    #return HF
    #return meanValue

In [ ]:
get

# The following is a code transformed into markdown
"""
n = 0
for i, row in FoundSth.iterrows():
    if n < 2:
        row.apply(getHF_each_box)
        #print(str(row['image_id']))
    n += 1
"""
#df = pd.DataFrame(columns=['HF1', 'HF2','HF3','HF4','HF5','HF6','HF7','HF8','HF9', 'HF10', 'HF11', 'HF12', 'HF13'])
#df = []
#get = FoundSth.loc[0:6]
#get = FoundSth.apply(getHF_each_box, axis = 1)
get.apply(getHF_each_box, axis = 1)
get.to_csv('FoundSthWithHF.csv')
#df


In [ ]:
# The following is a code transformed into markdown
"""
n = 0
for i, row in FoundSth.iterrows():
    if n < 2:
        row.apply(getHF_each_box)
        #print(str(row['image_id']))
    n += 1
"""
#df = pd.DataFrame(columns=['HF1', 'HF2','HF3','HF4','HF5','HF6','HF7','HF8','HF9', 'HF10', 'HF11', 'HF12', 'HF13'])
#df = []
#get = FoundSth.loc[0:6]
#get = FoundSth.apply(getHF_each_box, axis = 1)
get.apply(getHF_each_box, axis = 1)
get.to_csv('FoundSthWithHF.csv')
#df


In [ ]:
for i in df.index:
    print(df[i])

In [ ]:
#pd.DataFrame(df[2], columns = ['HF1', 'HF2','HF3','HF4','HF5','HF6','HF7','HF8','HF9', 'HF10', 'HF11', 'HF12', 'HF13'])


In [ ]:
get.at[2, "HF1"]

In [ ]:
#FoundSth.head()

In [ ]:
from pylab import gray, imshow, show

#img = img[100:300,100:300]
#imshow(img)
#show()
#h_feature = mh.features.haralick(img)
#print("Haralick Features")
#imshow(h_feature)
#show()
imgT = dicom2array(path=img_path)
#imgT = cv2.resize(imgT, None, fx=1/scale, fy=1/scale)
imgT = np.stack([imgT, imgT, imgT], axis=-1)
n = 0
"""
for box in boxes:
    n += 1
    if n == 1:
        print(box)i
        img = img[int(box[0]):int(box[2]), int(box[1]):int(box[3])]
        imshow(img)
        show()
        break
"""
for box in boxes:
    #print(box)
    img = imgT[int(box[1]):int(box[3]), int(box[0]):int(box[2])]
    meanValue = mh.features.haralick(img).mean(0)
    #print(meanValue)
    # After repeatedly try, I figure out the line above is correct, so I realize y should
    # go in front of x.
    #imshow(img)
    #show()

In [ ]:
#getHF(img_ids)

Set up a new dataset containing the HFs without images. This step is only for the training purpose right now.

In [ ]:
#train[train['image_id'] == img_id]
#classes[1]
check_single_disease = train.loc[(train['image_id'] == img_id) & (train['class_name'] == classes[5])]
#train.describe
train[train.index == 58645]
#check_single_disease

In [ ]:
from joblib import Parallel, delayed
from multiprocessing import cpu_count

In [ ]:
#def updateFoundSth():
#    for i, row in FoundSth.iterrows():
#    #print(f"{i}")
#        indx = int(f"{i}")
#        ret = getHF_each_box(indx)[0]
#        n = 0
#        for HF in HFs:
#            FoundSth.loc[FoundSth.index == indx, HF] = ret[n]
#            n += 1

In [ ]:
def updateFoundSth(i):
    #print(f"{i}")
    indx = int(f"{i}")
    ret = getHF_each_box(indx)[0]
    n = 0
    for HF in HFs:
        FoundSth.loc[FoundSth.index == indx, HF] = ret[n]
        n += 1

In [ ]:
#Parallel(n_jobs = int(cpu_count()), prefer = 'processes')(
#    delayed(getHF_each_box)(i)[0]
#    for i in FoundSth.index
#)

In [ ]:
FoundSth.head()

In [ ]:
#https://www.techbeamers.com/python-glob/
#The following code is used to generate DataFrame collectInfo
collectInfo = pd.DataFrame(columns = ['image_id', 'Health'])
for dico in glob.glob(f"{dataset_dir}/test/*.dicom"):
    str = dico.split("/")
    filename = str[-1].split(".")
    data = [{'image_id': filename[0], 'Health':True}]
    collectInfo = collectInfo.append(data,ignore_index=True,sort=False)

In [ ]:
#FoundSth.to_csv('FoundSthWithHF.csv')